In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import pickle
import os

/home/kapil/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def logistic_regression(X, y,n_classes,initializer=None,seed=42,learning_rate=0.01):
    n_inputs_including_bias = int(X.shape[1])
    if initializer is None:
        initializer = tf.random_uniform([n_inputs_including_bias, n_classes], -1.0, 1.0, seed=seed)
    theta = tf.Variable(initializer, name="theta")
    logits = tf.matmul(X, theta, name="logits")
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)+beta*tf.nn.l2_loss(theta))             
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.1
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,100000, 100.4, staircase=True)

# Passing global_step to minimize() will increment it at each step.
    training_op = (tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step))
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    #training_op = optimizer.minimize(loss)
    loss_summary = tf.summary.scalar('Cross-Entropy_Loss', loss)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    return loss, training_op, loss_summary, init, saver

In [3]:
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

In [4]:
def random_batch(X_train, y_train, batch_size):
    rnd_indices = np.random.randint(0, len(X_train), batch_size)
    X_batch = X_train[rnd_indices]
    y_batch = y_train[rnd_indices]
    return X_batch, y_batch

In [5]:
def add_biastomatrix(*args):
    matrices=[]
    for X in args:
        m=X.shape[0]
        X=np.c_[np.ones((m,1)),X]
        matrices.append(X)
    return tuple(matrices)

In [7]:
(X_train,X_test,X_devel)=add_biastomatrix(X_train,X_test,X_devel)

In [6]:
X_train=np.load("X_train.npy")
y_train=np.load("y_train.npy")
X_test=np.load("X_test.npy")
y_test=np.load("y_test.npy")
X_devel=np.load("X_devel.npy")
y_devel=np.load("y_devel.npy")

In [8]:
beta=0.001
n_inputs = 100
logdir = log_dir("logreg")
n_classes=y_train.shape[1]
X = tf.placeholder(tf.float32, shape=(None, n_inputs + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, n_classes), name="y")
loss, training_op, loss_summary, init, saver = logistic_regression(X, y,n_classes)
train_writer = tf.summary.FileWriter(logdir+"/train", tf.get_default_graph())
test_writer=tf.summary.FileWriter(logdir+"/test")
devel_writer=tf.summary.FileWriter(logdir+"/devel")
merged=tf.summary.merge_all()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
m=X_train.shape[0]
n_epochs = 1000
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
checkpoint_path = "/tmp/logistic_regressionl2.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./my_logreg_model"

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        # if the checkpoint file exists, restore the model and load the epoch number
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)
    
    for epoch in range(start_epoch, n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = random_batch(X_train, y_train, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        test_loss,summary_test = sess.run([loss,merged], feed_dict={X: X_test, y: y_test})
        test_writer.add_summary(summary_test, epoch)
        devel_loss,summary_devel=sess.run([loss,merged],feed_dict={X:X_devel,y:y_test})
        devel_writer.add_summary(summary_devel,epoch)
        if epoch % 50 == 0:
            train_loss,summary_train=sess.run([loss,merged],feed_dict={X:X_train,y:y_train})
            train_writer.add_summary(summary_train,epoch)
            print("Epoch:", epoch, "\tTest Loss:", test_loss,"\t Devel Loss:",
                  devel_loss,"\tTrain Loss:",train_loss)
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" % (epoch + 1))

    saver.save(sess, final_model_path)
    os.remove(checkpoint_epoch_path)

Epoch: 0 	Test Loss: 10.689052 	 Devel Loss: 10.684053 	Train Loss: 6.6586504
Epoch: 50 	Test Loss: 2.7236621 	 Devel Loss: 2.7239232 	Train Loss: 1.8424258
Epoch: 100 	Test Loss: 2.4849305 	 Devel Loss: 2.484696 	Train Loss: 1.7839105
Epoch: 150 	Test Loss: 2.462273 	 Devel Loss: 2.4607477 	Train Loss: 1.7780098
Epoch: 200 	Test Loss: 2.4443004 	 Devel Loss: 2.442893 	Train Loss: 1.7743899
Epoch: 250 	Test Loss: 2.453409 	 Devel Loss: 2.4537694 	Train Loss: 1.7758437
Epoch: 300 	Test Loss: 2.45224 	 Devel Loss: 2.4521554 	Train Loss: 1.7773219
Epoch: 350 	Test Loss: 2.44785 	 Devel Loss: 2.4478953 	Train Loss: 1.7754743
Epoch: 400 	Test Loss: 2.4288855 	 Devel Loss: 2.4264605 	Train Loss: 1.7766693
Epoch: 450 	Test Loss: 2.448495 	 Devel Loss: 2.4480827 	Train Loss: 1.7744725
Epoch: 500 	Test Loss: 2.4463577 	 Devel Loss: 2.4449086 	Train Loss: 1.7751057
Epoch: 550 	Test Loss: 2.4409719 	 Devel Loss: 2.4402215 	Train Loss: 1.7741535
Epoch: 600 	Test Loss: 2.4295514 	 Devel Loss: 2.428